In [2]:
# Import required libraries
import pandas as pd
import sqlite3
import os

# Install necessary packages
!pip install pandas

# CSV data sources:
# https://www.kaggle.com/datasets/nelgiriyewithana/countries-of-the-world-2023 - download the world-data-2023.csv
# https://www.kaggle.com/datasets/myrios/cost-of-living-index-by-country-by-number-2024 - download the Cost_of_Living_Index_by_Country_2024.csv
# Upload them to your Google Colab environment

In [3]:
# Check working directory to see if the files were uploaded correctly
print("Current working directory:", os.getcwd())
print("Files in the current directory:", os.listdir())

Current working directory: /content
Files in the current directory: ['.config', 'world-data-2023.csv', 'Cost_of_Living_Index_by_Country_2024.csv', 'sample_data']


In [6]:
def fetch_data(file_path):
    """
    Fetch data from a CSV file
    """
    if not os.path.exists(file_path):
        print(f"Error: The file {file_path} does not exist.")
        return None
    try:
        df = pd.read_csv(file_path)
        print(f"Successfully read CSV file from {file_path}")
        return df
    except Exception as e:
        print(f"An unexpected error occurred while reading the file: {e}")
        return None

def convert_format(df, output_format):
    """
    Convert DataFrame to specified output format
    """
    if output_format == 'json':
        return df.to_json(orient='records')
    elif output_format == 'csv':
        return df.to_csv(index=False)
    elif output_format == 'sql':
        return df
    else:
        raise ValueError("Invalid output format. Choose 'json', 'csv', or 'sql'.")

def modify_columns(df, columns_to_keep=None, columns_to_add=None):
    """
    Modify DataFrame columns based on user input
    """
    if columns_to_keep:
        df = df[columns_to_keep]
    if columns_to_add:
        df = df.assign(**columns_to_add)
    return df

def store_data(data, output_format, filename=None, table_name=None):
    """
    Store data in specified format (file or SQL database)
    """
    if output_format in ['json', 'csv']:
        with open(filename, 'w') as f:
            f.write(data)
        print(f"Data stored in {filename}")
    elif output_format == 'sql':
        with sqlite3.connect('output.db') as conn:
            data.to_sql(table_name, conn, if_exists='replace', index=False)
        print(f"Data stored in SQLite database 'output.db', table '{table_name}'")

def generate_summary(df, stage):
    """
    Generate and print summary of the data
    """
    print(f"\n{stage} Summary:")
    print(f"Number of records: {len(df)}")
    print(f"Number of columns: {len(df.columns)}")
    print(f"Columns: {', '.join(df.columns)}")

def clean_birth_rate_data(df):
    """
    Clean and transform birth rate data
    """
    columns_to_keep = ['Country', 'Birth Rate']
    df = df[columns_to_keep]
    df = df.dropna()
    df['Birth Rate'] = pd.to_numeric(df['Birth Rate'].astype(str).str.replace('[^\d.]', '', regex=True), errors='coerce')
    return df

def clean_cost_of_living_data(df):
    """
    Clean and transform cost of living data
    """
    columns_to_keep = ['Country', 'Cost of Living Index']
    df = df[columns_to_keep]
    df = df.dropna()
    df['Cost of Living Index'] = pd.to_numeric(df['Cost of Living Index'], errors='coerce')
    return df

In [7]:
def get_user_input():
    """
    Get user input for data source and output format
    """
    output_format = input("Enter output format (json/csv/sql): ").lower()
    return output_format

def get_column_modifications():
    """
    Get user input for column modifications
    """
    columns_to_keep = input("Enter columns to keep (comma-separated, leave blank for all): ").split(',') if input("Do you want to keep specific columns? (y/n): ").lower() == 'y' else None
    columns_to_add = {col: input(f"Enter value for new column '{col}': ") for col in input("Enter new columns to add (comma-separated, leave blank for none): ").split(',') if col}
    return columns_to_keep, columns_to_add

In [8]:
def main():
    """
    Main function to run the ETL pipeline
    """
    try:
        # Fetch data
        birth_rate_df = fetch_data("/content/world-data-2023.csv")
        cost_of_living_df = fetch_data("/content/Cost_of_Living_Index_by_Country_2024.csv")

        if birth_rate_df is None or cost_of_living_df is None:
            raise ValueError("Failed to fetch data")

        # Generate pre-processing summary
        generate_summary(birth_rate_df, "Birth Rate Pre-processing")
        generate_summary(cost_of_living_df, "Cost of Living Pre-processing")

        # Clean and transform data
        cleaned_birth_rate_df = clean_birth_rate_data(birth_rate_df)
        cleaned_cost_of_living_df = clean_cost_of_living_data(cost_of_living_df)

        # Merge datasets
        merged_df = pd.merge(cleaned_birth_rate_df, cleaned_cost_of_living_df, on='Country', how='inner')

        # Get user input
        output_format = get_user_input()

        # Modify columns
        columns_to_keep, columns_to_add = get_column_modifications()
        merged_df = modify_columns(merged_df, columns_to_keep, columns_to_add)

        # Convert format
        converted_data = convert_format(merged_df, output_format)

        # Store data
        if output_format in ['json', 'csv']:
            filename = f"/content/{input('Enter output filename: ')}"
            store_data(converted_data, output_format, filename)
        elif output_format == 'sql':
            table_name = input("Enter SQL table name: ")
            store_data(merged_df, output_format, table_name=table_name)

        # Generate post-processing summary
        generate_summary(merged_df, "Post-processing")

    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()

Successfully read CSV file from /content/world-data-2023.csv
Successfully read CSV file from /content/Cost_of_Living_Index_by_Country_2024.csv

Birth Rate Pre-processing Summary:
Number of records: 195
Number of columns: 35
Columns: Country, Density
(P/Km2), Abbreviation, Agricultural Land( %), Land Area(Km2), Armed Forces size, Birth Rate, Calling Code, Capital/Major City, Co2-Emissions, CPI, CPI Change (%), Currency-Code, Fertility Rate, Forested Area (%), Gasoline Price, GDP, Gross primary education enrollment (%), Gross tertiary education enrollment (%), Infant mortality, Largest city, Life expectancy, Maternal mortality ratio, Minimum wage, Official language, Out of pocket health expenditure, Physicians per thousand, Population, Population: Labor force participation (%), Tax revenue (%), Total tax rate, Unemployment rate, Urban_population, Latitude, Longitude

Cost of Living Pre-processing Summary:
Number of records: 121
Number of columns: 8
Columns: Rank, Country, Cost of Living 